## Reproduction of Table 4

This notebook reproduces the values in Table 4 (project-specific reproduction performance). 

In [1]:
import sys
sys.path.append("../../scripts/")
import json
import pandas

In [2]:
with open('../../data/Defects4J/all_d4j_crashes.txt') as f:
    jcrashpack_bugs = [e.strip().replace('-', '_') for e in f.readlines()]

with open('../../data/Defects4J/invalid_bug_reports.txt') as f:
    invalid_bugs = [e.strip().replace('-', '_') for e in f.readlines()]

In [3]:
with open('../../data/Defects4J/all_d4j_crashes.txt') as f:
    jcrashpack_bugs = [e.strip().replace('-', '_') for e in f.readlines()]

with open('../../data/Defects4J/invalid_bug_reports.txt') as f:
    invalid_bugs = [e.strip().replace('-', '_') for e in f.readlines()]

def evaluate(raw_result, only_crash=False):
    rows = []
    
    for bug_id, test_exec_results in raw_result.items():
        if bug_id in invalid_bugs:
            continue
        if only_crash and bug_id not in jcrashpack_bugs:
            continue
            
        for i, (fname, res) in enumerate(test_exec_results.items()):
            javalang_parse_error = False
            is_compile_error = False
            is_runtime_error = False
            buggy_version_failing = False
            fixed_version_failing = False
            success = False

            # Fall 1: Das ganze Ergebnis ist nur ein String (Parsing-Fehler des LLM-Outputs)
            if isinstance(res, str):
                javalang_parse_error = True 
            else:
                # Sichere Referenzen holen
                buggy_res = res.get('buggy')
                fixed_res = res.get('fixed')

                # Fall 2: 'buggy' oder 'fixed' sind Strings (Fehlermeldungen) oder None
                # Das passiert, wenn dein Runner abstürzt oder Timeouts hat.
                if not isinstance(buggy_res, dict) or not isinstance(fixed_res, dict):
                    is_runtime_error = True
                
                # Fall 3: Alles ist sauber gelaufen, wir haben Dictionaries
                elif buggy_res['compile_error'] or fixed_res['compile_error']:
                    is_compile_error = True
                elif buggy_res['runtime_error'] or fixed_res['runtime_error']:
                    is_runtime_error = True
                else:
                    # Logikprüfung
                    if buggy_res['autogen_failed']:
                        buggy_version_failing = True
                    if fixed_res['autogen_failed']:
                        fixed_version_failing = True
                    
                    if buggy_version_failing and (not fixed_version_failing):
                        success = True
            
            rows.append({
                'project': bug_id.split('_')[0],
                'bug_id': bug_id,
                'test_no': i+1,
                'javalang_parse_error': javalang_parse_error,
                'is_compile_error': is_compile_error,
                'is_runtime_error': is_runtime_error,
                'buggy_version_failing': buggy_version_failing,
                'fixed_version_failing': fixed_version_failing,
                'fname': fname,
                'success': success,
            })
    
    return pandas.DataFrame(rows)

EXPERIMENTS = ["repro", "basic", "exp1", "exp2", "exp3", "exp4", "all_in"]

for exp in EXPERIMENTS:

    RESULT_PATH = f'/vol/fob-vol7/mi21/kunzmart/Master-Thesis/results/{exp}_bereinigt.json'


    with open(RESULT_PATH) as f:
        reproduction_df = evaluate(json.load(f))

    aggr_eval_df = reproduction_df.groupby('bug_id').sum().reset_index()
    aggr_eval_df['project'] = aggr_eval_df.bug_id.apply(lambda x: x.split('_')[0])
    succeeded_bugs = aggr_eval_df[aggr_eval_df.success > 0]

    total_succ = 0
    total_bugs = 0
    tab_results = []
    for project in sorted(set(succeeded_bugs.project.tolist())):
        succeeded_bug_ids = succeeded_bugs[succeeded_bugs.project == project].bug_id.tolist()
        succ_num = len(succeeded_bug_ids)
        total_succ += succ_num
        total_num = (aggr_eval_df.project == project).sum()
        total_bugs += total_num
        tab_results.append({'Project': project, 'rep': succ_num, 'total': total_num})
    tab_results.append({'Project': 'Total', 'rep': total_succ, 'total': total_bugs})

    tab_df = pandas.DataFrame(tab_results)
    print(f"\n\nResults for {exp}:\n{tab_df}")



Results for repro:
            Project  rep  total
0             Chart    2      7
1               Cli   12     29
2           Closure    2    172
3             Codec    8     18
4       Collections    1      4
5          Compress    6     46
6               Csv    6     16
7              Gson    4     11
8       JacksonCore    3     24
9   JacksonDatabind   11    107
10            Jsoup   40     92
11           JxPath    4     19
12             Lang   19     63
13             Math   14     36
14            Total  132    644


Results for basic:
           Project  rep  total
0              Cli    1     28
1            Codec    1     18
2              Csv    1     16
3      JacksonCore    1     23
4  JacksonDatabind    2    107
5            Jsoup    5     92
6             Lang    3     63
7             Math    4    104
8             Time    3     23
9            Total   21    474


Results for exp1:
            Project  rep  total
0               Cli    2     29
1             Codec  

In [7]:
    RESULT_PATH = f'/vol/fob-vol7/mi21/kunzmart/Master-Thesis/results/repro.json'


    with open(RESULT_PATH) as f:
        reproduction_df = evaluate(json.load(f))

    aggr_eval_df = reproduction_df.groupby('bug_id').sum().reset_index()
    aggr_eval_df['project'] = aggr_eval_df.bug_id.apply(lambda x: x.split('_')[0])
    succeeded_bugs = aggr_eval_df[aggr_eval_df.success > 0]

    total_succ = 0
    total_bugs = 0
    tab_results = []
    for project in sorted(set(succeeded_bugs.project.tolist())):
        succeeded_bug_ids = succeeded_bugs[succeeded_bugs.project == project].bug_id.tolist()
        succ_num = len(succeeded_bug_ids)
        total_succ += succ_num
        total_num = (aggr_eval_df.project == project).sum()
        total_bugs += total_num
        tab_results.append({'Project': project, 'rep': succ_num, 'total': total_num})
    tab_results.append({'Project': 'Total', 'rep': total_succ, 'total': total_bugs})

    tab_df = pandas.DataFrame(tab_results)
    tab_df

,Project,rep,total
0,Chart,2,7
1,Cli,12,29
2,Closure,2,172
3,Codec,8,18
4,Collections,1,4
5,Compress,6,46
6,Csv,6,16
7,Gson,4,11
8,JacksonCore,3,24
9,JacksonDatabind,11,107


In [5]:
    RESULT_PATH = "/vol/fob-vol7/mi21/kunzmart/Master-Thesis/results/jxpath_repro_cleaned.json"


    with open(RESULT_PATH) as f:
        reproduction_df = evaluate(json.load(f))

    aggr_eval_df = reproduction_df.groupby('bug_id').sum().reset_index()
    aggr_eval_df['project'] = aggr_eval_df.bug_id.apply(lambda x: x.split('_')[0])
    succeeded_bugs = aggr_eval_df[aggr_eval_df.success > 0]

    total_succ = 0
    total_bugs = 0
    tab_results = []
    for project in sorted(set(succeeded_bugs.project.tolist())):
        succeeded_bug_ids = succeeded_bugs[succeeded_bugs.project == project].bug_id.tolist()
        succ_num = len(succeeded_bug_ids)
        total_succ += succ_num
        total_num = (aggr_eval_df.project == project).sum()
        total_bugs += total_num
        tab_results.append({'Project': project, 'rep': succ_num, 'total': total_num})
    tab_results.append({'Project': 'Total', 'rep': total_succ, 'total': total_bugs})

    tab_df = pandas.DataFrame(tab_results)
    print(f"\n\nResults for {exp}:\n{tab_df}")

FileNotFoundError: [Errno 2] No such file or directory: '/vol/fob-vol7/mi21/kunzmart/Master-Thesis/results/jxpath_repro_cleaned.json'

In [ ]:
RESULT_PATH = '../../results/all_in_GHRB.json'

In [ ]:
with open(RESULT_PATH) as f:
    reproduction_df = evaluate(json.load(f))

aggr_eval_df = reproduction_df.groupby('bug_id').sum().reset_index()
aggr_eval_df['project'] = aggr_eval_df.bug_id.apply(lambda x: x.split('_')[0])
succeeded_bugs = aggr_eval_df[aggr_eval_df.success > 0]

total_succ = 0
total_bugs = 0
tab_results = []
for project in sorted(set(succeeded_bugs.project.tolist())):
    succeeded_bug_ids = succeeded_bugs[succeeded_bugs.project == project].bug_id.tolist()
    succ_num = len(succeeded_bug_ids)
    total_succ += succ_num
    total_num = (aggr_eval_df.project == project).sum()
    total_bugs += total_num
    tab_results.append({'Project': project, 'rep': succ_num, 'total': total_num})
tab_results.append({'Project': 'Total', 'rep': total_succ, 'total': total_bugs})

tab_df = pandas.DataFrame(tab_results)
tab_df

In [ ]:
RESULT_PATH = '../../../results/d4j_repro_02.json'

with open(RESULT_PATH) as f:
    reproduction_df = evaluate(json.load(f))

aggr_eval_df = reproduction_df.groupby('bug_id').sum().reset_index()
aggr_eval_df['project'] = aggr_eval_df.bug_id.apply(lambda x: x.split('_')[0])
succeeded_bugs = aggr_eval_df[aggr_eval_df.success > 0]

total_succ = 0
total_bugs = 0
tab_results = []
for project in sorted(set(succeeded_bugs.project.tolist())):
    succeeded_bug_ids = succeeded_bugs[succeeded_bugs.project == project].bug_id.tolist()
    succ_num = len(succeeded_bug_ids)
    total_succ += succ_num
    total_num = (aggr_eval_df.project == project).sum()
    total_bugs += total_num
    tab_results.append({'Project': project, 'rep': succ_num, 'total': total_num})
tab_results.append({'Project': 'Total', 'rep': total_succ, 'total': total_bugs})

tab_df = pandas.DataFrame(tab_results)
print(RESULT_PATH)
tab_df

In [ ]:
#RESULT_PATH = '../../results/postprocess_d4j_default_run.json'
#RESULT_PATH = '../../../results/d4j_repro.json'
RESULT_PATH = '../../../results/d4j_all_in_02.json'


In [ ]:
with open(RESULT_PATH) as f:
    reproduction_df = evaluate(json.load(f))

aggr_eval_df = reproduction_df.groupby('bug_id').sum().reset_index()
aggr_eval_df['project'] = aggr_eval_df.bug_id.apply(lambda x: x.split('_')[0])
succeeded_bugs = aggr_eval_df[aggr_eval_df.success > 0]

total_succ = 0
total_bugs = 0
tab_results = []
for project in sorted(set(succeeded_bugs.project.tolist())):
    succeeded_bug_ids = succeeded_bugs[succeeded_bugs.project == project].bug_id.tolist()
    succ_num = len(succeeded_bug_ids)
    total_succ += succ_num
    total_num = (aggr_eval_df.project == project).sum()
    total_bugs += total_num
    tab_results.append({'Project': project, 'rep': succ_num, 'total': total_num})
tab_results.append({'Project': 'Total', 'rep': total_succ, 'total': total_bugs})

tab_df = pandas.DataFrame(tab_results)
tab_df

In [ ]:
RESULT_PATH = '../../../results/d4j_basic_02.json'

with open(RESULT_PATH) as f:
    reproduction_df = evaluate(json.load(f))

aggr_eval_df = reproduction_df.groupby('bug_id').sum().reset_index()
aggr_eval_df['project'] = aggr_eval_df.bug_id.apply(lambda x: x.split('_')[0])
succeeded_bugs = aggr_eval_df[aggr_eval_df.success > 0]

total_succ = 0
total_bugs = 0
tab_results = []
for project in sorted(set(succeeded_bugs.project.tolist())):
    succeeded_bug_ids = succeeded_bugs[succeeded_bugs.project == project].bug_id.tolist()
    succ_num = len(succeeded_bug_ids)
    total_succ += succ_num
    total_num = (aggr_eval_df.project == project).sum()
    total_bugs += total_num
    tab_results.append({'Project': project, 'rep': succ_num, 'total': total_num})
tab_results.append({'Project': 'Total', 'rep': total_succ, 'total': total_bugs})

tab_df = pandas.DataFrame(tab_results)
print(RESULT_PATH)
tab_df

In [ ]:
RESULT_PATH = '../../../results/d4j_exp1_02.json'

with open(RESULT_PATH) as f:
    reproduction_df = evaluate(json.load(f))

aggr_eval_df = reproduction_df.groupby('bug_id').sum().reset_index()
aggr_eval_df['project'] = aggr_eval_df.bug_id.apply(lambda x: x.split('_')[0])
succeeded_bugs = aggr_eval_df[aggr_eval_df.success > 0]

total_succ = 0
total_bugs = 0
tab_results = []
for project in sorted(set(succeeded_bugs.project.tolist())):
    succeeded_bug_ids = succeeded_bugs[succeeded_bugs.project == project].bug_id.tolist()
    succ_num = len(succeeded_bug_ids)
    total_succ += succ_num
    total_num = (aggr_eval_df.project == project).sum()
    total_bugs += total_num
    tab_results.append({'Project': project, 'rep': succ_num, 'total': total_num})
tab_results.append({'Project': 'Total', 'rep': total_succ, 'total': total_bugs})

tab_df = pandas.DataFrame(tab_results)
print(RESULT_PATH)
tab_df

In [ ]:
RESULT_PATH = '../../../results/d4j_repro_02.json'

with open(RESULT_PATH) as f:
    reproduction_df = evaluate(json.load(f))

aggr_eval_df = reproduction_df.groupby('bug_id').sum().reset_index()
aggr_eval_df['project'] = aggr_eval_df.bug_id.apply(lambda x: x.split('_')[0])
succeeded_bugs = aggr_eval_df[aggr_eval_df.success > 0]

total_succ = 0
total_bugs = 0
tab_results = []
for project in sorted(set(succeeded_bugs.project.tolist())):
    succeeded_bug_ids = succeeded_bugs[succeeded_bugs.project == project].bug_id.tolist()
    succ_num = len(succeeded_bug_ids)
    total_succ += succ_num
    total_num = (aggr_eval_df.project == project).sum()
    total_bugs += total_num
    tab_results.append({'Project': project, 'rep': succ_num, 'total': total_num})
tab_results.append({'Project': 'Total', 'rep': total_succ, 'total': total_bugs})

tab_df = pandas.DataFrame(tab_results)
print(RESULT_PATH)
tab_df

In [ ]:
RESULT_PATH = '../../../results/d4j_exp3_02.json'

with open(RESULT_PATH) as f:
    reproduction_df = evaluate(json.load(f))

aggr_eval_df = reproduction_df.groupby('bug_id').sum().reset_index()
aggr_eval_df['project'] = aggr_eval_df.bug_id.apply(lambda x: x.split('_')[0])
succeeded_bugs = aggr_eval_df[aggr_eval_df.success > 0]

total_succ = 0
total_bugs = 0
tab_results = []
for project in sorted(set(succeeded_bugs.project.tolist())):
    succeeded_bug_ids = succeeded_bugs[succeeded_bugs.project == project].bug_id.tolist()
    succ_num = len(succeeded_bug_ids)
    total_succ += succ_num
    total_num = (aggr_eval_df.project == project).sum()
    total_bugs += total_num
    tab_results.append({'Project': project, 'rep': succ_num, 'total': total_num})
tab_results.append({'Project': 'Total', 'rep': total_succ, 'total': total_bugs})

tab_df = pandas.DataFrame(tab_results)
print(RESULT_PATH)
tab_df

In [ ]:
RESULT_PATH = '../../results/all_in_GHRB.json'

with open(RESULT_PATH) as f:
    reproduction_df = evaluate(json.load(f))

aggr_eval_df = reproduction_df.groupby('bug_id').sum().reset_index()
aggr_eval_df['project'] = aggr_eval_df.bug_id.apply(lambda x: x.split('_')[0])
succeeded_bugs = aggr_eval_df[aggr_eval_df.success > 0]

total_succ = 0
total_bugs = 0
tab_results = []
for project in sorted(set(succeeded_bugs.project.tolist())):
    succeeded_bug_ids = succeeded_bugs[succeeded_bugs.project == project].bug_id.tolist()
    succ_num = len(succeeded_bug_ids)
    total_succ += succ_num
    total_num = (aggr_eval_df.project == project).sum()
    total_bugs += total_num
    tab_results.append({'Project': project, 'rep': succ_num, 'total': total_num})
tab_results.append({'Project': 'Total', 'rep': total_succ, 'total': total_bugs})

tab_df = pandas.DataFrame(tab_results)
print(RESULT_PATH)
tab_df

In [ ]:
RESULT_PATH = '../../../results/d4j_all_in_02.json'

with open(RESULT_PATH) as f:
    reproduction_df = evaluate(json.load(f))

aggr_eval_df = reproduction_df.groupby('bug_id').sum().reset_index()
aggr_eval_df['project'] = aggr_eval_df.bug_id.apply(lambda x: x.split('_')[0])
succeeded_bugs = aggr_eval_df[aggr_eval_df.success > 0]

total_succ = 0
total_bugs = 0
tab_results = []
for project in sorted(set(succeeded_bugs.project.tolist())):
    succeeded_bug_ids = succeeded_bugs[succeeded_bugs.project == project].bug_id.tolist()
    succ_num = len(succeeded_bug_ids)
    total_succ += succ_num
    total_num = (aggr_eval_df.project == project).sum()
    total_bugs += total_num
    tab_results.append({'Project': project, 'rep': succ_num, 'total': total_num})
tab_results.append({'Project': 'Total', 'rep': total_succ, 'total': total_bugs})

tab_df = pandas.DataFrame(tab_results)
print(RESULT_PATH)
tab_df

In [ ]:
import json
import pandas as pd
import os

# --- KONFIGURATION ---
# Der Pfad zu deiner Datei
file_path = '/vol/fob-vol7/mi21/kunzmart/Master-Thesis/libro/results/example2_n50_GHRB.json'

# --- HILFSFUNKTIONEN ---

def evaluate(raw_result, only_crash=False):
    rows = []
    
    # Da wir keine externen Listen haben, filtern wir hier nicht
    
    for bug_id, test_exec_results in raw_result.items():
        # Falls test_exec_results selbst null ist (sollte nicht passieren, aber zur Sicherheit)
        if test_exec_results is None:
            continue
            
        for i, (fname, res) in enumerate(test_exec_results.items()):
            javalang_parse_error = False
            is_compile_error = False
            is_runtime_error = False
            buggy_version_failing = False
            fixed_version_failing = False
            success = False
            
            # 1. Fall: Das Ergebnis ist null (passiert bei Abstürzen des Tools)
            if res is None:
                is_runtime_error = True
            
            # 2. Fall: Das Ergebnis ist ein String (Parse Error)
            elif isinstance(res, str):
                javalang_parse_error = True 
            
            # 3. Fall: Das Ergebnis ist ein Dictionary (Normaler Ablauf)
            else:
                # Sicherer Zugriff auf 'buggy' und 'fixed' mit .get(), falls keys fehlen oder null sind
                buggy_res = res.get('buggy')
                fixed_res = res.get('fixed')

                # BUGGY Check
                if buggy_res:
                    if buggy_res.get('compile_error'): is_compile_error = True
                    elif buggy_res.get('runtime_error'): is_runtime_error = True
                    elif buggy_res.get('autogen_failed'): buggy_version_failing = True
                
                # FIXED Check (Hier lag der Fehler: fixed kann null sein)
                if fixed_res:
                    if fixed_res.get('compile_error'): is_compile_error = True
                    elif fixed_res.get('runtime_error'): is_runtime_error = True
                    elif fixed_res.get('autogen_failed'): fixed_version_failing = True
                
                # Erfolgslogik: 
                # 1. Buggy muss fehlschlagen (autogen_failed = True)
                # 2. Fixed darf NICHT fehlschlagen (autogen_failed = False)
                # 3. Fixed darf nicht None sein (sonst wissen wir nicht, ob es geklappt hat)
                if buggy_version_failing and (not fixed_version_failing) and (fixed_res is not None):
                    success = True
            
            # Projektname extrahieren (z.B. "checkstyle" aus "checkstyle_checkstyle-10825")
            try:
                project_name = bug_id.split('_')[0]
            except:
                project_name = "unknown"
            
            rows.append({
                'project': project_name,
                'bug_id': bug_id,
                'test_no': i+1,
                'javalang_parse_error': javalang_parse_error,
                'is_compile_error': is_compile_error,
                'is_runtime_error': is_runtime_error,
                'buggy_version_failing': buggy_version_failing,
                'fixed_version_failing': fixed_version_failing,
                'fname': fname,
                'success': success,
            })
    
    return pd.DataFrame(rows)

# --- HAUPTPROGRAMM ---

try:
    print(f"Lese Datei von: {file_path} ...")
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Datei nicht gefunden: {file_path}")

    with open(file_path, 'r') as f:
        raw_result = json.load(f)
    
    print(f"Datei geladen. {len(raw_result)} Bugs gefunden. Starte Auswertung...")

    # Auswertung durchführen
    reproduction_df = evaluate(raw_result)

    if reproduction_df.empty:
        print("Keine Daten zum Auswerten gefunden.")
    else:
        # Aggregation
        aggr_eval_df = reproduction_df.groupby('bug_id')[['success']].sum().reset_index()
        
        # Projektname sicher extrahieren
        aggr_eval_df['project'] = aggr_eval_df.bug_id.apply(lambda x: x.split('_')[0] if '_' in x else x)

        succeeded_bugs = aggr_eval_df[aggr_eval_df.success > 0]

        total_succ = 0
        total_bugs = 0
        tab_results = []

        unique_projects = sorted(set(aggr_eval_df.project.tolist()))

        for project in unique_projects:
            succ_num = len(succeeded_bugs[succeeded_bugs.project == project])
            total_succ += succ_num
            
            total_num = (aggr_eval_df.project == project).sum()
            total_bugs += total_num
            
            tab_results.append({'Project': project, 'rep': succ_num, 'total': total_num})

        tab_results.append({'Project': 'Total', 'rep': total_succ, 'total': total_bugs})

        tab_df = pd.DataFrame(tab_results)
        
        # Berechnung der Prozentzahl hinzufügen
        tab_df['Rate'] = (tab_df['rep'] / tab_df['total'] * 100).fillna(0).round(1).astype(str) + '%'

        print("\nResults:")
        print(tab_df.to_string(index=False))

except FileNotFoundError as e:
    print(f"\nFEHLER: {e}")
except json.JSONDecodeError:
    print(f"\nFEHLER: Die Datei '{file_path}' ist keine gültige JSON-Datei.")
except Exception as e:
    print(f"\nEin unerwarteter Fehler ist aufgetreten:")
    import traceback
    traceback.print_exc()